In [1]:
import camelot
import pandas as pd
import numpy as np
import re

In [2]:
first_table = camelot.read_pdf('kentucky1.pdf')

In [3]:
second_table = camelot.read_pdf("kentucky2.pdf")

In [4]:
# vertically concatenate both tables
df = pd.concat([first_table[0].df, second_table[0].df]).reset_index(drop = True)

In [5]:
# set the first row to be the column headers
df.columns = df.iloc[0]

# drop the first row (which is now the header)
df = df[1:].reset_index(drop = True)
df

0    Sample  Collected                                     Received Location  \
0   AQ03020  7/15/2019                7/16/2019 Ashland Water Works            
1   AQ02922   7/8/2019                         7/9/2019 Augusta WTP            
2   AQ04874  10/1/2019                       10/1/2019 Barbourville            
3   AQ03210  7/22/2019          7/23/2019 Bardstown Municipal Water            
4   AQ03216  7/22/2019         7/23/2019 Beaver Dam Municipal Water            
..      ...        ...                                          ...      ...   
76  AQ03646  8/13/2019                    8/14/2019 Trapps Water Co            
77  AQ03658  8/13/2019                8/14/2019 Trimble County WD#1            
78  AQ04362  9/10/2019     9/10/2019 Vanceburg Electric Plant Board            
79  AQ04031  8/27/2019  8/28/2019 Water Service Corp of KY  Clinton            
80  AQ03575  8/12/2019     8/13/2019 Winchester Municipal Utilities            

0  Type   PFBS HFPO- DA  PFHpA  PFHxS  ADONA   PFOA   PFOS   PFNA Units  \
0    SW   0.00    18.30   0.00   0.00   0.00   4.74   1.96   0.00  ng/L   
1    GW   1.88     0.00   0.00   0.00   0.00   4.43   1.00   0.00  ng/L   
2    SW   0.00     0.00   0.00   0.00   0.00   1.39   1.03   0.00  ng/L   
3    SW   0.00     0.00   0.00   0.00   0.00   0.00   0.00   0.00  ng/L   
4    GW   0.00     0.00   0.00   0.00   0.00   0.00   0.00   0.00  ng/L   
..  ...    ...      ...    ...    ...    ...    ...    ...    ...   ...   
76   GW  0.000    0.000  0.000  0.000  0.000  0.000  0.000  0.000  ng/L   
77   GW  0.000    0.000  0.000  0.000  0.000  0.000  0.000  0.000  ng/L   
78   GW  0.000    0.000  0.000  0.000  0.000  0.000  1.400  0.000  ng/L   
79   GW  0.000    0.000  0.000  0.000  0.000  0.000  0.000  0.000  ng/L   
80   SW  0.000    0.000  0.000  0.000  0.000  0.000  1.690  0.000  ng/L   

0       LATITUDE_M                        LONGITUDE_ Aquifer/Source  \
0   38.45277800000            -82.61305600000 Ohio R                  
1   38.77361100000   -84.01944400000 Ohio R Alluvium                  
2   36.86444400000  -83.88194400000 Cumberland River                  
3   37.81055600000        -85.50583300000 Buffalo Cr                  
4   37.40250000000      -86.87750000000 Breathitt SS                  
..             ...                               ...            ...   
76  38.98416700000   -84.82861100000 Ohio R Alluvium                  
77  38.53944400000   -85.40277800000 Ohio R Alluvium                  
78  38.59840195720   -83.27048732530 Ohio R Alluvium                  
79  36.67161843140         -88.99627626600 Claiborne                  
80  37.94722200000       -84.22833300000 L Howard Cr                  

0  River Basin          Aquifer General Land Use  
0         Ohio                             urban  
1                                   ORA    rural  
2   Cumberland                             urban  
3         Salt                             rural  
4               Pennsylvanian Sandstone    urban  
..         ...                      ...      ...  
76                                  ORA    rural  
77                                  ORA    rural  
78                                  ORA    rural  
79                Mississippi Embayment    rural  
80    Kentucky                             rural  

[81 rows x 20 columns]

In [6]:
for i in np.arange(0, df.shape[0]):
    # grab that row
    row = df.iloc[i]
    
    # fix the location and date of each row 
    date_location_text = row["Received"]
    
    if date_location_text == "":
        date_location_text = row["Location"]
        
    date_loc_match = re.match(r'(\d{1,2}/\d{1,2}/\d{4}) (.+)', date_location_text)
    
    date = date_loc_match.group(1)
    location = date_loc_match.group(2)
    
    # fix aquifer + coordinate issue
    coord_source_text = row["LONGITUDE_"]
    df.at[i, "Received"] = date
    df.at[i, "Location"] = location

    # fix the aquifier/source and longitude
    coord_source_match = re.match(r'([-+]?\d+\.\d+)\s+(.+)', coord_source_text)

    coordinate = coord_source_match.group(1)
    address = coord_source_match.group(2)

    df.at[i, "LONGITUDE_"] = coordinate
    df.at[i, "Aquifer/Source"] = address

In [7]:
# rename some columns 
df.rename(columns={'LATITUDE_M': 'Latitude', 'LONGITUDE_': 'Longitude'}, inplace=True)

In [8]:
df.to_csv("kentucky.csv")